In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite:///database.sqlite")

df = pd.read_sql_table("football_data", engine)

# df.info(memory_usage="deep")

In [9]:
df.head()

,Season,Date,Div,Country,League,Referee,HomeTeam,AwayTeam,FTHG,FTAG,...,SOH,SYA,SYD,SYH,VCA,VCD,VCH,WHA,WHD,WHH
0,2018/2019,2018-10-03,E1,England,Championship,O Langford,Sheffield Weds,West Brom,2,2,...,NaN,NaN,NaN,NaN,2.10,3.6,3.60,2.10,3.5,3.50
1,2018/2019,2018-10-03,E1,England,Championship,G Ward,Rotherham,Bristol City,0,0,...,NaN,NaN,NaN,NaN,2.10,3.5,3.75,2.15,3.4,3.50
2,2018/2019,2018-10-03,E1,England,Championship,J Moss,Nott'm Forest,Millwall,2,2,...,NaN,NaN,NaN,NaN,4.30,3.6,1.93,4.00,3.6,1.91
3,2018/2019,2018-10-03,E1,England,Championship,G Eltringham,Derby,Norwich,1,1,...,NaN,NaN,NaN,NaN,3.60,3.5,2.15,3.40,3.4,2.15
4,2018/2019,2018-10-03,E1,England,Championship,R Jones,Blackburn,Sheffield United,0,2,...,NaN,NaN,NaN,NaN,2.63,3.3,2.90,2.55,3.3,2.80


In [3]:
print("Number of records: {}".format(df.shape[0]))
print("Number of features: {}".format(df.shape[1]))

Number of records: 132380
Number of features: 110


In [4]:
print("Unique values for goal-related features:")
print("FTHG: {}".format(df["FTHG"].unique()))
print("FTAG: {}".format(df["FTAG"].unique()))
print("FTR: {}".format(df["FTR"].unique()))
print("HTHG: {}".format(df["HTHG"].unique()))
print("HTAG: {}".format(df["HTAG"].unique()))
print("HTR: {}".format(df["HTR"].unique()))

Unique values for goal-related features:
FTHG: [ 2  0  1  3  5  4  7  6  8  9 10 -1]
FTAG: [ 2  0  1  3  4  5  6  7  8  9 -1]
FTR: ['D' 'A' 'H' None]
HTHG: [ 2  0  1  3  4  5 -1  6  7]
HTAG: [ 0  1  2  4  3 -1  6  5]
HTR: ['H' 'D' 'A' None]


We can see some issues with the data:
1. FTHG, FTAG, HTHG and HTAG can have negative values
2. FTR and HTR have missing data

If all of FTHG, FTAG, HTHG, and HTAG of a match are negative, we may consider removing that match since it's not possible to determine the outcome of the match

In [5]:
mask = (df["FTHG"] == -1) & (df["FTAG"] == -1) & (df["HTHG"] == -1) & (df["HTAG"] == -1)
print("Number of matches where all of FTHG, FTAG, HTHG and HTAG are negative: {}".format(df[mask].shape[0]))

Number of matches where all of FTHG, FTAG, HTHG and HTAG are negative: 1


In [6]:
# Remove the record that has all negative values
i = df[mask].index[0]
df.drop(index = i, inplace = True)
df = df.reset_index()

For the remaining records that have HTHG and HTAG equal to -1, we can try setting them to 0 (this is an arbitary decision). Also, set the HTR (Half time result) of these matches to "D"

In [7]:
mask = (df["HTHG"] == -1) & (df["HTAG"] == -1)
df.loc[mask, "HTHG"] = 0
df.loc[mask, "HTAG"] = 0
df.loc[mask, "HTR"] = "D"

In [8]:
null_count = df.isnull().sum()
null_count /= df.shape[0]

print("Number of features that have at least 40% missing values: {}".format((null_count >= 0.4).sum()))

Number of features that have at least 40% missing values: 58


In [9]:
null_count_df = null_count.to_frame().reset_index()
null_count_df.columns = ["Feature", "Null_perct"]
null_count_df.sort_values("Null_perct", ascending = False, inplace = True)

# Keep only the features where the % of missing values is less than 40%
features_lt_40_missing = null_count_df[null_count_df["Null_perct"] < 0.4]["Feature"]
reduced_df = df[features_lt_40_missing]

In [10]:
reduced_df.columns

Index(['GBA', 'GBD', 'GBH', 'VCD', 'VCH', 'VCA', 'BbMxAHA', 'BbAvAHA',
       'BbMxAHH', 'BbAvAHH', 'BbAHh', 'BbAH', 'BbMx<2.5', 'BbAv<2.5',
       'BbMx>2.5', 'BbAv>2.5', 'BbOU', 'BbAvA', 'BbMxA', 'BbMxD', 'Bb1X2',
       'BbAvH', 'BbAvD', 'BbMxH', 'BWH', 'BWD', 'BWA', 'B365D', 'B365A',
       'B365H', 'LBD', 'LBH', 'LBA', 'WHA', 'WHD', 'WHH', 'IWA', 'IWH', 'IWD',
       'AwayTeam', 'HomeTeam', 'Date', 'Div', 'Country', 'League', 'FTAG',
       'FTHG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Season', 'index'],
      dtype='object')

In [11]:
# Let's try building some models without odds data first since they have more missing values
df_game_stats_only = reduced_df.loc[:, ['AwayTeam', 'HomeTeam', 'HTR', 'FTR', 'Date', 'HTAG', 'HTHG', 'FTAG', 'FTHG',
                                 'League', 'Country', 'Div', 'Season']]

In [12]:
df_game_stats_only.isnull().sum()

AwayTeam    1046
HomeTeam    1046
HTR            0
FTR            0
Date           0
HTAG           0
HTHG           0
FTAG           0
FTHG           0
League         0
Country        0
Div            0
Season         0
dtype: int64

Remove games where HomeTeam and AwayTeam are null since they only account for about 1% of the dataset

In [13]:
df_game_stats_only.dropna(subset=["HomeTeam", "AwayTeam"], inplace=True)